In [2]:
import httpx
import pandas as pd
import datetime as dt
import sqlite3
import json  
import time
import darts
from darts import TimeSeries

In [3]:
# obtained from fuzzworks
invData = pd.read_csv('../invTypes.csv')

In [5]:
def getMarketPrices(inplace=False):
    print('Fetching market prices')
    r = httpx.get(
        'https://esi.evetech.net/latest/markets/prices/?datasource=tranquility',)
    if(r.status_code == 200):
        df = pd.read_json(r.text)
        df['date'] = dt.datetime.now()
        con = sqlite3.connect('../data/data.db')
        df.to_sql(con=con, name="market_data", if_exists="append")
        con.close()
        if(inplace):
            return(df)
        # cur = con.cursor()
        # cur.execute('select * from market_data')
        # results = cur.fetchall()
        # print(results)
    else:
        print(f"Request did not succeed! {r.status_code}")
        rjson = json.loads(r.text)
        if(rjson['timeout'] != None or 0):
            time.sleep((rjson['timeout']/1000))
getMarketPrices(inplace=True)
# adjusted_price, average_price, type_id

Fetching market prices


,adjusted_price,average_price,type_id,date
0,0.000000e+00,24379193.55,43691,2022-12-14 19:13:20.637223
1,1.106410e+06,1504030.04,32772,2022-12-14 19:13:20.637223
2,7.189439e+04,68254.69,32774,2022-12-14 19:13:20.637223
3,5.746279e+06,7416848.51,32780,2022-12-14 19:13:20.637223
4,6.316466e+00,5.45,32782,2022-12-14 19:13:20.637223
...,...,...,...,...
14046,0.000000e+00,39499722.22,46914,2022-12-14 19:13:20.637223
14047,0.000000e+00,1584.49,44212,2022-12-14 19:13:20.637223
14048,0.000000e+00,1114.96,55645,2022-12-14 19:13:20.637223
14049,0.000000e+00,6456700.00,46916,2022-12-14 19:13:20.637223


In [ ]:
def getItemPrices(id):
    # refresh database with new data
    getMarketPrices()
    con = sqlite3.connect('../data.db')
    # df = pd.read_sql_table(table_name='market_data',con=con,parse_dates='date')
    # cur = con.cursor()
    # cur.execute("select * from market_data")
    df = pd.read_sql('select * from market_data', con, parse_dates=['date'])
    selected_df = df.loc[df['type_id'] == id]
    if(len(selected_df) != 0):
        return(selected_df)
    else:
        print('Sorry, either this is not a valid item id or something went wrong. :P')

In [ ]:
def getIDInfo(id):
    selected_row = invData.loc[invData['typeID'] == id]
    return selected_row
# getIDInfo(32772)

In [ ]:
def getIdFromName(name):
    # NOTE: This function could be improved with a better search engine
    selected_rows = invData[invData['typeName'].str.contains(name, False)]
    selected_rows = selected_rows[['typeName', 'typeID']]
    return(selected_rows)
# getIdFromName('Booster')

In [ ]:
# def getHistoricalData():
     # Jita region id: 30000142
# update to the rant below some shitter was calling invalid ID types causing server load, so they shut down the endpoint indefinitely for now, fucking dumb ass's
# This is makes me very angry, apparently the historical market data call doesn't work so I have to utilize time and bandwidth information to build my own db    